In [129]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import html5lib
import matplotlib as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn import decomposition
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

import requests
from bs4 import BeautifulSoup
import csv
import re # regular expressions library

%matplotlib inline



In [72]:
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/reprocessed.hungarian.data'

webpage = requests.get(data_url)

webcontents = webpage.text # this returns the webpage html content

#print(data)
list = []
list=re.split(r'\n', webcontents)
datalist = []
data = []
for i in range(len(list)):
    datalist = re.split(r'\s+', list[i])
    data.append(datalist)
    #print(datalist)

#print(len(data[0]))

In [68]:
name_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/heart-disease.names'
webpage = requests.get(name_url)
contents = webpage.text
#print(contents)

In [44]:
#-- 1. #3  (age) 
matched = []
matched = re.findall(r'--\s\d+\.\s\#\d+\s+\(\w+\)', contents)
taglist = []
for i in range(len(matched)):
    m = re.match(r"--\s\d+\.\s\#\d+\s+\((\w+)\)", matched[i])
    print(i,m.group(1) )
    taglist.append(m.group(1))

0 age
1 sex
2 cp
3 trestbps
4 chol
5 fbs
6 restecg
7 thalach
8 exang
9 oldpeak
10 slope
11 ca
12 thal
13 num


In [63]:
print(type(taglist))
df = pd.DataFrame.from_records(data, columns=taglist)
print(df.head())

<class 'list'>
  age sex cp trestbps chol fbs restecg thalach exang oldpeak slope  ca thal  \
0  40   1  2      140  289   0       0     172     0       0    -9  -9   -9   
1  49   0  3      160  180   0       0     156     0       1     2  -9   -9   
2  37   1  2      130  283   0       1      98     0       0    -9  -9   -9   
3  48   0  4      138  214   0       0     108     1     1.5     2  -9   -9   
4  54   1  3      150   -9   0       0     122     0       0    -9  -9   -9   

  num  
0   0  
1   1  
2   0  
3   3  
4   0  


(296, 14)

In [66]:
##drop na
df = df.dropna(axis=0, how='any')
df.shape

(294, 14)

In [114]:
feature_cols = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
X = df[feature_cols]
X = preprocessing.scale(X)
y = df['num']
#print(X[0:10])

In [115]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=5)

In [137]:
##pick 11 because the ratio is over 95%
pca = decomposition.PCA(n_components=11)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

[ 0.22910972  0.12209178  0.09912614  0.09583057  0.08519139  0.07705091
  0.07016747  0.06000009  0.04813019  0.03786461  0.03706467]
[ 0.22910972  0.3512015   0.45032764  0.54615821  0.6313496   0.70840051
  0.77856798  0.83856807  0.88669826  0.92456287  0.96162754]


In [138]:
my_SVM = SVC(C=1, kernel='rbf', gamma=0.1,random_state=5)
my_SVM.fit(X_train_pca, y_train)
y_predict_svm= my_SVM.predict(X_test_pca)
score_svm = accuracy_score(y_test, y_predict_svm)
print(score_svm)


0.779661016949


In [132]:
my_ANN = MLPClassifier(hidden_layer_sizes=(100,20), activation= 'logistic', 
                       solver='adam', alpha=1e-5, random_state=5, 
                       learning_rate_init = 0.02)
my_ANN.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 20), learning_rate='constant',
       learning_rate_init=0.02, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=5, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [131]:
y_predict_ann = my_ANN.predict(X_test)
score_ann = accuracy_score(y_test, y_predict_ann)
print(score_ann)

0.728813559322
